# Qwen3 Resume Parser Fine-Tuning with LoRA

This notebook fine-tunes Qwen3-0.6B base model for resume parsing using LoRA (Low-Rank Adaptation) for efficient fine-tuning.

**Features:**
- LoRA fine-tuning for memory efficiency
- Supervised Fine-Tuning (SFT) using TRL's SFTTrainer
- Optimized hyperparameters for ~5k samples
- Google Colab GPU support


## 1. Environment Setup and GPU Check


In [1]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"✓ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA version: {torch.version.cuda}")
    print(f"✓ GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will be slow on CPU.")
    print("   Please enable GPU in Colab: Runtime → Change runtime type → GPU")


✓ GPU available: NVIDIA L4
✓ CUDA version: 12.6
✓ GPU memory: 22.16 GB


In [2]:
# Install required packages
!pip install -q transformers peft trl datasets accelerate bitsandbytes

print("✓ Packages installed successfully")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.9 MB/s eta 0:00:00
✓ Packages installed successfully


## 2. Imports


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import os

print("✓ Imports successful")


✓ Imports successful


## 3. Configuration


In [4]:
# Model configuration
MODEL_NAME = "Qwen/Qwen3-0.6B"

# Dataset configuration
DATASET_PATH = "/home/qwen3_training_dataset.jsonl"  # Path to your converted dataset
# If uploading to Colab, upload the JSONL file and update the path

# Output configuration
OUTPUT_DIR = "/home/qwen3-resume-parser-lora"
FINAL_MODEL_DIR = "/home/qwen3-resume-parser-lora-final"

print(f"Model: {MODEL_NAME}")
print(f"Dataset: {DATASET_PATH}")
print(f"Output directory: {OUTPUT_DIR}")


Model: Qwen/Qwen3-0.6B
Dataset: /home/qwen3_training_dataset.jsonl
Output directory: /home/qwen3-resume-parser-lora


## 4. Load Model and Tokenizer


In [5]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Set padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✓ Tokenizer loaded")

print("\nLoading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Use float16 for memory efficiency
    device_map="auto",  # Automatically handle device placement
    trust_remote_code=True
)

print("✓ Model loaded")
print(f"Model device: {next(model.parameters()).device}")
print(f"Model dtype: {next(model.parameters()).dtype}")


Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✓ Model loaded
Model device: cuda:0
Model dtype: torch.float16


## 5. Configure LoRA


In [6]:
# LoRA configuration following best practices
lora_config = LoraConfig(
    r=8,  # Rank - controls adapter capacity
    lora_alpha=16,  # Scaling factor (typically 2x rank)
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Qwen3 attention modules
    lora_dropout=0.05,  # Dropout for regularization
    bias="none",  # Don't train bias terms
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA to model
print("Applying LoRA configuration...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()
print("\n✓ LoRA configuration applied")


Applying LoRA configuration...
trainable params: 2,293,760 || all params: 598,343,680 || trainable%: 0.3834

✓ LoRA configuration applied


## 6. Load Dataset


In [7]:
print(f"Loading dataset from {DATASET_PATH}...")

# Load JSONL dataset
dataset = load_dataset("json", data_files=DATASET_PATH)

# The dataset already has "text" field from conversion script
# No additional preprocessing needed

print(f"✓ Dataset loaded")
print(f"  Train examples: {len(dataset['train'])}")
print(f"  Sample keys: {list(dataset['train'][0].keys())}")

# Show a sample (first 500 chars)
if len(dataset['train']) > 0:
    sample_text = dataset['train'][0]['text']
    print(f"\n  Sample text (first 500 chars):")
    print(f"  {sample_text[:500]}...")


Loading dataset from /home/qwen3_training_dataset.jsonl...


Generating train split: 0 examples [00:00, ? examples/s]

✓ Dataset loaded
  Train examples: 4879
  Sample keys: ['text']

  Sample text (first 500 chars):
  <|im_start|>system
You are an expert resume parser. Extract structured information from resumes and return ONLY valid JSON. Do not include explanations or extra text.<|im_end|>
<|im_start|>user
Resume:
ADULT EDUCATION INSTRUCTOR
Summary
Seasoned Agriculture Teacher with more than 20 years of experience in this world of education. Excellent teaching and leadership skills. Track
record of achieving exceptional results in not only FFA programs but also Credit Recovery Programs at my current high sc...


In [29]:
import torch
torch.cuda.empty_cache()
print("GPU cache cleared")

GPU cache cleared


## 7. Configure Training


In [8]:
# Training hyperparameters optimized for ~5k samples
# Based on Hugging Face SmolLM3 SFT guide

BATCH_SIZE =2  # Reduced batch size
GRADIENT_ACCUMULATION_STEPS = 8  # Increased to maintain effective batch size
LEARNING_RATE = 1e-4
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 1536
WARMUP_STEPS = 100

# Calculate total steps
total_steps = (len(dataset['train']) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)) * NUM_EPOCHS

print("Training Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"  Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  Warmup steps: {WARMUP_STEPS}")
print(f"  Estimated total steps: ~{total_steps}")

# Configure training arguments
# In newer TRL versions, use max_length instead of max_seq_length
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    warmup_steps=WARMUP_STEPS,
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    fp16=True,  # Mixed precision training for memory efficiency
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    report_to="none",  # Set to "wandb" or "tensorboard" if you want logging
    push_to_hub=False,  # Set to True if pushing to HuggingFace Hub
    remove_unused_columns=False,
    gradient_checkpointing=True, # Enable gradient checkpointing for memory efficiency
    # SFT-specific parameters
    dataset_text_field="text",  # Field containing the formatted text
    max_length=MAX_SEQ_LENGTH,  # Use max_length instead of max_seq_length
    packing=False,  # Don't pack sequences - each example is separate
)

print("\n✓ Training configuration created")

Training Configuration:
  Batch size: 2
  Gradient accumulation: 8
  Effective batch size: 16
  Learning rate: 0.0001
  Epochs: 3
  Max sequence length: 1536
  Warmup steps: 100
  Estimated total steps: ~912

✓ Training configuration created


## 8. Initialize Trainer


In [9]:
# Initialize SFTTrainer
# In newer TRL versions, dataset_text_field, max_seq_length, and packing are in SFTConfig
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    args=training_args,
)

print("✓ Trainer initialized")
print(f"  Training examples: {len(dataset['train'])}")
print(f"  Output directory: {OUTPUT_DIR}")


Adding EOS to train dataset:   0%|          | 0/4879 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4879 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/4879 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


✓ Trainer initialized
  Training examples: 4879
  Output directory: /home/qwen3-resume-parser-lora


## 9. Start Training


In [10]:
# Start training
print("Starting training...")
print("="*70)

trainer.train()

print("="*70)
print("✓ Training completed!")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Starting training...


Step,Training Loss
50,3.090200
100,2.726200
150,2.601500
200,2.569000
250,2.536200
300,2.514500
350,2.499400
400,2.471500
450,2.478000
500,2.479000


✓ Training completed!


## 10. Save Final Model


In [11]:
# Save final model
print(f"Saving final model to {FINAL_MODEL_DIR}...")

trainer.save_model(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print(f"✓ Model saved to {FINAL_MODEL_DIR}")
print("\nTo download the model from Colab:")
print(f"  - Use Files panel on the left")
print(f"  - Or run: !zip -r {FINAL_MODEL_DIR}.zip {FINAL_MODEL_DIR}")


Saving final model to /home/qwen3-resume-parser-lora-final...
✓ Model saved to /home/qwen3-resume-parser-lora-final

To download the model from Colab:
  - Use Files panel on the left
  - Or run: !zip -r /home/qwen3-resume-parser-lora-final.zip /home/qwen3-resume-parser-lora-final


In [13]:
!zip -r /home/qwen3-resume-parser-lora-final.zip /home/qwen3-resume-parser-lora-final

  adding: home/qwen3-resume-parser-lora-final/ (stored 0%)
  adding: home/qwen3-resume-parser-lora-final/README.md (deflated 65%)
  adding: home/qwen3-resume-parser-lora-final/chat_template.jinja (deflated 76%)
  adding: home/qwen3-resume-parser-lora-final/tokenizer_config.json (deflated 90%)
  adding: home/qwen3-resume-parser-lora-final/special_tokens_map.json (deflated 69%)
  adding: home/qwen3-resume-parser-lora-final/training_args.bin (deflated 53%)
  adding: home/qwen3-resume-parser-lora-final/adapter_model.safetensors (deflated 7%)
  adding: home/qwen3-resume-parser-lora-final/added_tokens.json (deflated 68%)
  adding: home/qwen3-resume-parser-lora-final/tokenizer.json (deflated 81%)
  adding: home/qwen3-resume-parser-lora-final/adapter_config.json (deflated 58%)
  adding: home/qwen3-resume-parser-lora-final/vocab.json (deflated 61%)
  adding: home/qwen3-resume-parser-lora-final/merges.txt (deflated 57%)


## 11. Inference Example


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import json

print("Loading fine-tuned model for inference...")

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, FINAL_MODEL_DIR)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(FINAL_MODEL_DIR, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✓ Fine-tuned model loaded")

example_resume = """Senior IT Project Manager/Consultant Senior <span class=\"hl\">IT</span> <span class=\"hl\">Project</span> <span class=\"hl\">Manager</span>/Consultant Senior IT Project Manager/Consultant Chicago, IL Work Experience Senior IT Project Manager/Consultant Northern Trust Bank - Chicago, IL March 2015 to August 2015 Full PM responsibilities as indicated by Northern Trust's SDLC requirements, using waterfall methodology.  Managed multiple medium-sized IDM/IAM projects from Initiation through Design, including Program Management. Personal Leave September 2014 to February 2015 Senior IT Project Manager/Consultant Northern Trust Bank - Chicago, IL March 2012 to August 2014 Full PM responsibilities as indicated by Northern Trust's SDLC requirements, using waterfall methodology.  Managed a multi-year, multi-phased, large SDLC Project: Led project from Initiation to Project Closure as BA and PM.  IDM Rel 1.6 Web Cash Movement: development for auto-provisioning Web Cash Movement and all other web applications via Identity Management System (IDM/IAM) and Access Certification, delivering the functionality for Remove, Updates for Employees, Contractors and Clients, as well as Rehiring Terminated Employees and Contractors.  IDM Rel 1.4 Web Arch for UAT: enablement for auto-provisioning of Web Cash Movement apps via IDM to UAT Web Arch environment.  IDM Rel 1.3 Web Admin Alternative: UAT application auto-provisioning for Web Cash Movement via IDM, delivering initial functionality for Add Access to Production for Employees, Contractors and Clients.  Managed many small SDLC projects:  Manitou Patch Update: update to security system to current patch. Initiation.  IDM Rel 1.8 Employee Transfer. Initiation to Planning. Certifications/Licenses Certified Green Belt in Six Sigma Quality/Process Improvement Present Certified in ITIL Foundations, Operations and Service Delivery, ITIL V3 Present Certified Project Management Professional (PMP) PMI PMP #1345786 July 2010 to July 2016 Advanced training in project management, DePaul University, Chicago, IL Present Extensive corporate training in project management and PMO Present Additional Information HIGHLIGHTS of QUALIFICATIONS  ? Accomplished PMP certified IT Project Manager with 20+ years of project management experience at top tier global companies. ? IT professional with extensive background in Operations and Service Delivery, Business & Systems Development, and consulting. ? Highly skilled in adapting interpersonal skills with existing SMEs and support teams, project teams, and C-level management. ? Proven ability in achieving significant cost reductions, revenue growth, and efficiency improvements through the intelligent application of technology. ? Manages multiple IT projects simultaneously while meeting scope, deadlines and budgets. ? Strong track record of delivering application solutions (application development projects) to complex business problems in Identity Access Management (IAM), retail applications, IT infrastructure, and custom application software. ? Strategically focuses on high level goals while managing the details. ? Expert in leading decentralized teams of IT professionals to deliver excellence. ? Highly self-motivated Project Manager with strong team building skills.  Project Management Expertise  Developed and maintained productive working relationships with Stakeholders, Business Owners, Project Sponsors, Business Analysts, Solution Architects, Vendors, and Key Clients  Gained Stakeholder trust and brokered consensus throughout project  Acted as internal advocate for project  Provided insight for mission/business critical  Partnered between IT and Business to ensure internal teams are properly aligned throughout the project  Defined project charter, scope, objectives, deliverables and established measurement for benefits realization  Defined the project governance and provided regular status updates to steering committee in order to facilitate informed decisions  Gathered information required to estimate project cost, resources, and schedule  Developed and continually updated project plans and schedules  Managed the day-to-day activities of projects and staff to ensure project deliverables are on schedule and within cost performance.  Ensured the project deliverables meet users' requirements  Managed costs, schedule, scope control, change management and performance for the project  Developed a communication plan to convey project scope, goals, milestones, budget, risk, status, change requests and critical issues to the client and project team  Determined staffing requirements and developed budget requests for resources  Worked with Resource Managers in order to effectively align resources across projects  Negotiated changes in project resources as necessary to achieve objects and timelines  Created and managed resource calendars  Recruited and managed appropriate staffing resources  Drove all phases of IT Project Development life cycle including guiding the project team through requirements gathering, SIT and UAT planning and testing, and installation preparation and execution  Developed process designs to support business solutions and clarify business requirements  Approved invoices for payment  Identified the elements of risk in the project, utilizing SMEs, BA's, System Analysts, and other project personnel  Analyzed and prioritized project risks and assess potential impact to the client  Developed and maintained risk plans, processes and systems in order to mitigate risks and crate an action plan when they occur  Maintained issue lists and facilitated the resolution of issues throughout the project life cycle  Created and managed capital requests and revisions  Developed a detailed cost baseline from cost estimations  Refined project cost estimates and confirmed funding sources  Monitored and controlled the actual cost of the project versus the budget  Conducted in-depth root cause analysis of project budget discrepancies  Managed program release plan activities and communicated status to project team.  Performed project close financial processes  Adapted communication style to audience(s)  Wrote and presented status reports, Benefits Realization, Lessons Learned, Project Implementation Review and other project closure reports for and to management and steering committee  Implemented methodologies, processes, and tools developed by the PMO, exercising good judgment to tailor methodology to project  Owned SDLC process and internal processes, and trained and mentored staff and other PMs  Provided feedback regarding the continuous improvement of project standards and procedures  A driven, hands-on leader who does what it takes to get the job done.  COMPUTER SKILLS  Software: MS Office Suite, MS Outlook, MS Project, Visio Professional, SharePoint, SAP, XcelleNet, Siebel, Remedy, Planview Operating Systems: Windows (all), UNIX POS Systems: Retalix, Radiant, Wayne Nucleus, MSI, VeriFone, Gilbarco, Compris, SASI, NCR, IBM, ICL Programming: XML, SQL, Visual Basic, C++, JCL, HTML",

"""

messages = [
    {
        "role": "system",
        "content": (
            "You are an expert resume parser. "
            "Extract structured information from resumes and return ONLY valid JSON. "
            "Do not include explanations or extra text."
        )
    },
    {
        "role": "user",
        "content": f"Resume:\n{example_resume}"
    }
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,          # IMPORTANT
        pad_token_id=tokenizer.eos_token_id
    )

assistant_response = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True
).strip()

print("\nGENERATED RESPONSE:\n", assistant_response)

try:
    parsed = json.loads(assistant_response)
    print("\n✓ Valid JSON")
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError as e:
    print("\n⚠️ Invalid JSON:", e)


Loading fine-tuned model for inference...
✓ Fine-tuned model loaded

GENERATED RESPONSE:
 {"current_title": "Senior IT Project Manager/Consultant", "previous_titles": ["Senior IT Project Manager/Consultant", "Senior IT Project Manager/Consultant"], "current_company": "Northern Trust Bank", "previous_companies": ["Northern Trust Bank", "Northern Trust Bank"], "years_experience": 20, "seniority": "senior", "primary_domain": "IT Project Management", "industries": ["Banking", "IT Infrastructure", "IT Consulting"], "core_skills": ["Project Management", "ITIL", "Project Management Professional (PMP)"], "secondary_skills": ["Process Design", "Risk Management", "Team Building"], "tools": ["MS Office Suite", "MS Project", "SharePoint"], "leadership_experience": true, "key_achievements": ["Managed multiple medium-sized IDM/IAM projects from Initiation through Design, including Program Management", "Led a multi-year, multi-phased, large SDLC Project (IDM Rel 1.6 Web Cash Movement) and managed man